In [1]:
import quimb as qu
import quimb.tensor as qtn
# import cotengra as ctg

# opt = ctg.ReusableHyperOptimizer(
#     reconf_opts={},
#     max_repeats=16,
#     parallel=True,
#     # use the following for persistently cached paths
#     # directory='ctg_path_cache',
# )

import networkx as nx

reg = 3
n = 18
seed = 666
G = nx.random_regular_graph(reg, n, seed=seed)

terms = {(i, j): 1 for i, j in G.edges}
p = 1
gammas = qu.randn(p)
betas = qu.randn(p)
gg = 1
jj = 1
circ_ex = qtn.circ_qaoa(terms, p, gammas, betas,jj,gg)
circ_ex.psi.draw(color=['PSI0', 'H', 'RZZ', 'RX'])

AttributeError: module 'quimb.tensor' has no attribute 'circ_qaoa'

In [ ]:
circ_ex.get_rdm_lightcone_simplified([0]).draw(color=['PSI0', 'H', 'RZZ', 'RX'], highlight_inds=['k0', 'b0'])

In [ ]:
import tqdm

localham = -jj * ( (qu.pauli('Z') & qu.pauli('Z')) + (gg * (qu.pauli('I') & qu.pauli('X'))))

local_exp_rehs = [
    circ_ex.local_expectation_rehearse(weight * localham, edge)
    for edge, weight in tqdm.tqdm(list(terms.items()))
]

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
ax1.plot([rehs['W'] for rehs in local_exp_rehs], color='green')
ax1.set_ylabel('contraction width, $W$, [log2]', color='green')
ax1.tick_params(axis='y', labelcolor='green')

ax2 = ax1.twinx()
ax2.plot([rehs['C'] for rehs in local_exp_rehs], color='orange')
ax2.set_ylabel('contraction cost, $C$, [log10]', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

In [ ]:
def energy(x):
    p = 1
    gammas = x[:p]
    betas = x[p:]
    circ = qtn.circ_qaoa(terms, p, gammas, betas,jj,gg)

    localham = -jj * ( (qu.pauli('Z') & qu.pauli('Z')) + (gg * (qu.pauli('I') & qu.pauli('X'))))
    ens = [
        circ.local_expectation(weight * localham, edge)
        for edge, weight in terms.items()
    ]

    return sum(ens).real

from skopt import Optimizer
from skopt.plots import plot_convergence, plot_objective
eps = 1e-6
bounds = (
    [(0.0       , qu.pi )] * p +
    [(0.0 , qu.pi)] * p
)

bopt = Optimizer(bounds)
for i in tqdm.trange(100):
    x = bopt.ask()
    res = bopt.tell(x, energy(x))
    
plot_convergence(res);